<a href="https://colab.research.google.com/github/andreahorbach/SeminarMSV/blob/main/TR_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sitzung 5: Plagiatserkennung - Teil 2
Heute schauen wir uns an, wie man mit Ähnlichkeitsmethoden einen Plagiatserkenner trainiert. Wir bauen auf die Ähnlichkeitsmaße aus der letzten Sitzung auf.

**Schritt 1: String Similarity / Oberflächenähnlichkeit**
Dieser Code sollte Ihnen aus der letzten Sitzung bekannt vorkommen.

In [ ]:
!pip install -U strsimpy # installiere die notwendigen Pakete
!pip install pylcs

In [ ]:
from strsimpy.levenshtein import Levenshtein
from pylcs import lcs_string_length

levenshtein = Levenshtein()

# Beispiele, die Sie zum Austesten verändern können
s1 = 'Ein Beispiel'
s2 = 'Ein Beispöl'
s3 = 'Anna isst einen Apfel'
s4 = 'Anna ist eine Apfelsine.'

#Levenshtein distance
print(levenshtein.distance(s1, s2))
print(levenshtein.distance(s3, s4))

# Longest Common Substring
print(lcs_string_length(s1, s2))
print(lcs_string_length(s3, s4))


**Schritt 2: Semantic Similarity**
Auch diesen Code kennen Sie bereits.

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [ ]:
def compute_semantic_similarity(s1, s2):
    embeddings1 = model.encode([s1], convert_to_tensor=True)
    embeddings2 = model.encode([s2], convert_to_tensor=True)
    return util.cos_sim(embeddings1, embeddings2)[0][0]

print("{:.4f}".format(compute_semantic_similarity("Mary walks home.", "Mary goes to her house")))
print("{:.4f}".format(compute_semantic_similarity("Maria geht nach Hause.", "Maria fährt heim.")))

**Aufgabe 1 (von letzter Woche):** Nehmen Sie an, Sie möchten den ersten Abschnitt des Wikipediaartikels zum Thema Plagiat plagiieren. Er lautet "Ein Plagiat ist die Anmaßung fremder geistiger Leistungen. Dies kann sich auf die Übernahme fremder Texte oder anderer Darstellungen bzw. Ideen beziehen, z. B. Zeitungs-, Magazinartikel, Fotos, Filme, Tonaufnahmen, Musik sowie Erfindungen, Design, wissenschaftliche Erkenntnisse, Melodien oder beides gleichzeitig."

Probieren Sie, wie Sie diese beiden Sätze so umformen können, dass durch Ihre Methoden möglichst nicht mehr direkt als Plagiat erkennbar sind. Probieren Sie verschiedene Optionen aus und beobachten Sie, wie sich die Werte der Metriken verändern.

**Schritt 3:** "To be or not to be that is the question" schrieb Shakespeare. Finden Sie - für jedes der drei Maße - den ähnlichsten Satz bei Jane Austen. Ist der Satz ein Plagiat? Hier eine mögliche Lösung - ebenfalls von letzter Woche.

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg

austen1 = gutenberg.sents('austen-emma.txt')
shakespeare = "To be or not to be that is the question ."
maxSim = 0.0
bestSentence = "dummy"
for sent in austen1:
  text = " ".join(sent) # macht aus der Liste von Tokens wieder einen Text.
  sim = compute_semantic_similarity(shakespeare, text)
  if (sim > maxSim):
      print("new most similar sentence: ", sim, text)
      maxSim = sim
print("Final results: ", maxSim, bestSentence)


**Schritt 4:** Um nach "echten " Plagiaten suchen können, ergänzen wir das Jane Austen Corpus um einige plagiierte Versionen des Shakespear-Zitates. Außerdem merken wir uns in einem separaten Array für jeden Satz das Label als kein-Plagiat (K) oder Plagiat (P). (Sie erinnern sich, das haben wir beim Machine Learninig vor zwei Sitzungen auch schon so gemacht) Natürlich sind nur unsere Ergänzungen Plagiate.

In [ ]:
plagiate = ["Not to be or to be is an important question", "I am wondering whether one should be.", "Existence is very relevant", "Sein oder nichtsein, das ist hier die Frage"]
austen_sentences = [" ".join(sent) for sent in austen1]
gold_labels = ["K" for sent in austen1]
print(austen_sentences)
print(gold_labels)

**Aufgabe 2:** Ergänzen Sie 5 weitere Sätze, die Plagiate des Shakespearezitats sind mit dem Label P.

**Aufgabe 3:** Benutzen Sie den Algorithmus von oben und suchen Sie den ähnlichsten Satz. Ist er ein Plagiat?

**Aufgabe 4:** Als nächstes sollen Sie die Sätze nach ihrer Ähnlichkeit zum Originalzitat sortieren. Sie müssen sich dafür also jede Ähnlichkeit zusammen mit dem Satz merken. Führen Sie den Code aus und testen Sie was passiert. TODO: plus Anpassungen

**Aufgabe 5:** Setzen Sie einen Threshold bei 0.9 und berechnen Sie Precision und Recall.

**Aufgabe 6:**  Finden Sie den Threshold, der den F-Score maximiert.